In [3]:
# Some constraints.
function puzzle_words(w1, idx1, w2, idx2)
    # println("$idx1 at $w1 is $(w1[idx1])")
    # println("$idx2 at $w2 is $(w2[idx2])")
    # println()
    return w1[idx1] == w2[idx2]
end

function diff(w1, w2)
    return w1 != w2
end

function queen_diff(w1, col1, w2, col2)
    # println("abs($(w1.row)-$(w2.row)) == abs($col1-$col2) -> $(abs(w1.row-w2.row) == abs(col1-col2))")
    return !(col1 == col2 || abs(w1.row-w2.row) == abs(col1-col2))
end

queen_diff (generic function with 1 method)

In [31]:
# Some helper structs 
mutable struct word
    name
    value
    d::Vector{Any}
    d_copy::Vector{Any}
end
mutable struct Queen
    name
    row
    col
    d::Vector{Any}
end
mutable struct Queen_DVFC
    name
    row
    col
    d::Vector{Any}
    d_copy::Vector{Any}
end
mutable struct R
    v1::word
    idx1
    v2::word
    idx2
end 
mutable struct R_queen
    v1::Queen
    v2::Queen
end 
mutable struct R_queen_DVFC
    v1::Queen_DVFC
    v2::Queen_DVFC
end 

In [36]:
# Some helper functions.
function _set_constrains!(w1::word, idx1, w2::word, idx2, Rs::Dict{Vector{word}, R})
    Rs[[w1,w2]] = R(w1, idx1, w2, idx2)
    Rs[[w2,w1]] = R(w2, idx2, w1, idx1)
    return nothing
end

function _set_constrains!(w1::Queen, w2::Queen, Rs::Dict{Vector{Queen}, R_queen})
    Rs[[w1,w2]] = R_queen(w1, w2)
    Rs[[w2,w1]] = R_queen(w2, w1)
    return nothing
end

function _set_constrains!(w1::Queen_DVFC, w2::Queen_DVFC, Rs::Dict{Vector{Queen_DVFC}, R_queen_DVFC})
    Rs[[w1,w2]] = R_queen_DVFC(w1, w2)
    Rs[[w2,w1]] = R_queen_DVFC(w2, w1)
    return nothing
end

function consistent(vars, Rs, D_copy, a, b, k, i)
    for j = 1 : i-1
        if !diff(vars[j].value, b)
            return false
        end
        R = get(Rs, [vars[j], vars[k]], nothing)
        if R != nothing
            idx1 = R.idx1
            idx2 = R.idx2
            cons = puzzle_words(vars[j].value, idx1, b, idx2)
            if !cons
                return false
            end
        end
    end
    if !diff(a, b)
        return false
    end
    R = get(Rs, [vars[i], vars[k]], nothing)
    if R != nothing
        idx1 = R.idx1
        idx2 = R.idx2
        cons = puzzle_words(a, idx1, b, idx2)
        if !cons
            return false
        end
    end
    return true
end

function print_tree(logs, order)
    strs = []
    dict = similar(order)
    for i = 1 : length(order)
        dict[i] = 2*i-1
        push!(strs, "")
        temp = ""
        for j = 1 : length("$(vars[order[i]].name):   ")
            temp *= " "
        end
        push!(strs, temp)
    end
    first_elim = true
    for log in logs
        op, up, down, value = log
        up_idx, down_idx = dict[up], dict[down]
        up_up_idx = up_idx - 2
        l = fld(length(value), 2)
        
        if up_up_idx > 0
            ct = 0
            for i = sizeof(strs[up_up_idx]) : -1 : 1
                try
                    curr = strs[up_up_idx][i]
                    if curr != ' '
                        break
                    end
                    ct += 1
                catch e
                    break
                end
            end
            strs[up_up_idx] = chop(strs[up_up_idx], tail = ct)
            for i = 1:ct
                strs[up_up_idx] *= '\u2501'
            end
        end
        
        if op == "elim"
            l += 1
            for i = 1 : up_up_idx - 1
                for j = 1 : 2*l+1
                    strs[i] *= " "
                end
            end
            if up_up_idx > 0
                for i = 1 : 2*l+1
                    strs[up_up_idx] *= '\u2501'
                    strs[up_up_idx+1] *= " "
                end
            end
            if first_elim
                for i = 1:l
                    strs[up_idx] *= " "
                end
                    strs[up_idx] *= '\u250C'
                for i = 1:l
                    strs[up_idx] *= '\u2504'
                end
                first_elim = false
            else
                for i = 1:l
                    strs[up_idx] *= '\u2504'
                end
                strs[up_idx] *= '\u252C'
                for i = 1:l
                    strs[up_idx] *= '\u2504'
                end
            end
            for i = up_idx+1 : down_idx-1
                for j = 1:l
                    strs[i] *= " "
                end
                strs[i] *= '\u250A'
                for j = 1:l
                    strs[i] *= " "
                end
            end
            strs[down_idx] *= "[$value]"
            for i = 1: (2*l-1-length(value))
                strs[down_idx] *= " "
            end
            for i = down_idx+1 : 12
                for j = 1 : 2*l+1
                    strs[i] *= " "
                end
            end
        else
            first_elim = true
            if up_up_idx > 0
                for i = 1:l
                    strs[up_up_idx] *= '\u2501'
                    strs[up_up_idx+1] *= " "
                end
                    strs[up_up_idx] *= '\u2513'
                    strs[up_up_idx+1] *= '\u2503'
                for i = 1:l
                    strs[up_up_idx] *= " "
                    strs[up_up_idx+1] *= " "
                end
            end
            strs[up_idx] *= "$value"
            for i = 1: (2*l+1-length(value))
                strs[up_idx] *= " "
            end
            for i = up_idx+1 : 12
                for j = 1 : 2*l+1
                    strs[i] *= " "
                end
            end
        end
    end 
    for i = 1 : length(order)
        println("$(vars[order[i]].name):   $(strs[2*i-1])")
        println(strs[2*i])
    end
    println("***********************************")
end
# ["elim", 2, 3, "SUN"]
# ["select", 2, -1, "ALSO"]
# ["elim", 1, 3, "red"], ["elim", 1, 4, "red"], ["elim", 1, 7, "red"], ["select", 1, "red"]
# ["elim", 3, 7, "blue"], ["select", 3, -1, "blue"]

print_tree (generic function with 1 method)

Below is the implementation of **`SELECT-VALUE`**.

In [6]:
# Implementation of SELECT-VALUE
function SV(logs, vars, Rs, n, i, D_copy)
    D_copy_copy = deepcopy(D_copy)
    while length(D_copy[i]) > 0
        signal = true
        a = popfirst!(D_copy[i])
        if consistent(vars, Rs, D_copy, a, i)
            push!(logs, ["select", i, 1, a])
            return a
        else
            push!(logs, ["elim", i-1, i, a])
        end
    end
    return nothing
end

function consistent(vars, Rs, D_copy, a, i)
    for j = 1 : i-1
        R = get(Rs, [vars[j], vars[i]], nothing)
        if R != nothing
            if !queen_diff(vars[j], vars[j].col, vars[i], a)
                return false
            end
        end
    end
    return true
end

consistent (generic function with 2 methods)

Below is the implementation of **`BACKTRACKING`**.

In [15]:
# Implementation of BACKTRACKING
function BACKTRACKING(order, logs, vars, Rs)
    ct = 0
    n = length(vars)
    # Copy all domains
    D = []
    for var in vars
        push!(D, deepcopy(var.d))
    end
    D_copy = deepcopy(D)
    # Initialize variable counter
    i = 1
    while 1 <= i <= n
        ct += 1
        for j = 1:6
            println("\t\tDomain of $(vars[j].name) = $(D_copy[j])")
        end
        new_value = SV(logs, vars, Rs, n, i, D_copy)
        println("Step #$ct:\t$(vars[i].name) chose $new_value from $(D_copy[i])")
        if new_value == nothing # no value was returned
            i -= 1 # backtrack
            if i == 0
                return "inconsistent"
            end
        else
            vars[i].col = new_value
            i += 1 # step forward
            D_copy[i] = deepcopy(D[i])
        end
        print_tree(logs, order) # print tree
    end
    return "success!"
end

BACKTRACKING (generic function with 1 method)

Below is the implementation of **`SELECT-VALUE-FORWARD-CHECKING`**.

In [22]:
# Implementation of SELECT-VALUE-FORWARD-CHECKING
function SVFC(logs, vars, Rs, n, i, D_copy)
    D_copy_copy = deepcopy(D_copy)
    while length(D_copy[i]) > 0
        signal = true
        a = popfirst!(D_copy[i])
        eliminate_list = [] # for draw tree purpose
        k = i+1
        while k <= n
            j = 1
            while j <= length(D_copy[k])
                b = D_copy[k][j]
                # println(D_copy[k])
                # println("\t\t\t\t\ttesting $b in $(vars[k].name)")
                j += 1
                if !consistent(vars, Rs, D_copy, a, b, k, i)
                    push!(eliminate_list, [k,b]) # for draw tree purpose
                    deleteat!(D_copy[k], findfirst(x->x==b,D_copy[k]))
                    j -= 1
                end
            end

            if length(D_copy[k]) == 0
                for j = i+1 : n
                    D_copy[j] = D_copy_copy[j]
                end
                println("\t\t\tD_copy[$k] = 0 when $(vars[i].name) chooses $a")
                signal = false
                k = n+1
            end
            k += 1
        end
        
        # record log for draw tree purpose
        for elim in eliminate_list
            push!(logs, ["elim", i, elim[1], elim[2]])
        end
        push!(logs, ["select", i, 1, a])

        if signal
            return a
        end
    end
    return nothing
end

SVFC (generic function with 1 method)

In [55]:
function consistent_queen(vars, Rs, D_copy, a, b, k, i)
    for j = 1 : i-1
        if !diff(vars[j].col, b)
            return false
        end
        R = get(Rs, [vars[j], vars[k]], nothing)
        if R != nothing
            if !queen_diff(vars[j], vars[j].col, vars[k], b)
                return false
            end
        end
    end
    if !diff(a, b)
        return false
    end
    R = get(Rs, [vars[i], vars[k]], nothing)
    if R != nothing
        if !queen_diff(vars[i], vars[i].col, vars[k], b)
            return false
        end
    end
    return true
end

# Implementation of SELECT-VALUE-FORWARD-CHECKING
function SVFC_queen(order, logs, vars, Rs, n, i, D_copy)
    D_copy_copy = deepcopy(D_copy)
    while length(D_copy[order[i]]) > 0
        signal = true
        a = popfirst!(D_copy[order[i]])
        eliminate_list = [] # for draw tree purpose
        k = i+1
        while k <= n
            j = 1
            while j <= length(D_copy[order[k]])
                b = D_copy[order[k]][j]
                # println(D_copy[k])
                # println("\t\t\t\t\ttesting $b in $(vars[k].name)")
                j += 1
                if !consistent_queen(vars, Rs, D_copy, a, b, order[k], order[i])
                    push!(eliminate_list, [order[k], b]) # for draw tree purpose
                    deleteat!(D_copy[order[k]], findfirst(x->x==b,D_copy[order[k]]))
                    j -= 1
                end
            end

            if length(D_copy[k]) == 0
                for j = i+1 : n
                    D_copy[order[j]] = D_copy_copy[order[j]]
                end
                println("\t\t\tD_copy[$(order[k])] = 0 when $(vars[order[i]].name) chooses $a")
                signal = false
                k = n+1
            end
            k += 1
        end
        
        # record log for draw tree purpose
        for elim in eliminate_list
            push!(logs, ["elim", order[i], elim[1], elim[2]])
        end
        push!(logs, ["select", order[i], 1, a])

        if signal
            return a
        end
    end
    return nothing
end

SVFC_queen (generic function with 2 methods)

Below is the implementation of **`DYNAMIC-VALUE-FORWARD-CHECKING`**.

In [54]:
# Implementation of DYNAMIC-VALUE-FORWARD-CHECKING
function DVFC(order, logs, vars, Rs)
    ct = 0
    n = length(vars)
    # Copy all domains
    D = []
    for var in vars
        push!(D, deepcopy(var.d))
    end
    # Initialize variable counter
    i = 1
    s = find_future_variable_with_smallest_domain(order, i, n, D)
    # rearrange variables (swap x_i+1 and x_s)
    temp = order[i+1]
    order[i+1] = order[s]
    order[s] = temp 
    println("s = $s, so new order is $order")
    while 1 <= i <= n
        ct += 1
        for j = 1:6
            println("\t\tDomain of $(vars[order[j]].name) = $(D[order[j]])")
        end
        vars[order[i]].d_copy= []
        for j = i+1 : n
            push!(vars[order[i]].d_copy, deepcopy(D[order[j]]))
        end
        new_value = SVFC_queen(order, logs, vars, Rs, n, i, D)
        println("Step #$ct:\t$(vars[order[i]].name) chose $new_value from $(D[order[i]])")
        if new_value == nothing # no value was returned
            i -= 1 # backtrack
            if i == 0
                return "inconsistent"
            end
            # if i == 1
            #     logs = []
            # end
            # reset each D'_k
            for k = 1:i
                println("\t\t       \tDomain of $(vars[order[k]].name) = $(D[order[k]])")
            end
            for k = i+1 : n
                D[order[k]] = deepcopy(vars[order[i]].d_copy[order[k-i]])
                println("\t\tRestore the domain of $(vars[order[k]].name) to $(D[order[k]])")
            end

        else
            vars[order[i]].col = new_value
            if i < n
                s = find_future_variable_with_smallest_domain(order, i, n, D)
                # rearrange variables (swap x_i+1 and x_s)
                temp = order[i+1]
                order[i+1] = order[s]
                order[s] = temp 
                println("s = $s, so new order is $order")
            end
            i += 1 # step forward
        end
        print_tree(logs, order) # print tree
    end
    return "success!"
end

function find_future_variable_with_smallest_domain(order, i, n, D)
    s = i+1
    min_l = length(D[order[s]])
    for j = i+2 :n
        if length(D[order[j]]) < min_l
            min_l = length(D[order[j]])
            s = j
        end
    end
    return s
end

find_future_variable_with_smallest_domain (generic function with 2 methods)

Below is the implementation of **`GENERALIZED-LOOK-AHEAD`**.

In [9]:
# Implementation of GENERALIZED-LOOK-AHEAD
function GLA(order, logs, vars, Rs, instantiate_method)
    ct = 0
    n = length(vars)
    # Copy all domains
    D = []
    for var in vars
        push!(D, deepcopy(var.d))
    end
    # Initialize variable counter
    i = 1
    while 1 <= i <= n
        ct += 1
        for j = 1:6
            println("\t\tDomain of $(vars[j].name) = $(D[j])")
        end
        vars[i].d_copy= []
        for j = i+1 : n
            push!(vars[i].d_copy, deepcopy(D[j]))
        end
        new_value = instantiate_method(logs, vars, Rs, n, i, D)
        println("Step #$ct:\t$(vars[i].name) chose $new_value from $(D[i])")
        if new_value == nothing # no value was returned
            i -= 1 # backtrack
            if i == 0
                return "inconsistent"
            end
            # if i == 1
            #     logs = []
            # end
            # reset each D'_k
            for k = 1:i
                println("\t\t       \tDomain of $(vars[k].name) = $(D[k])")
            end
            for k = i+1 : n
                D[k] = deepcopy(vars[i].d_copy[k-i])
                println("\t\tRestore the domain of $(vars[k].name) to $(D[k])")
            end

        else
            vars[i].value = new_value
            i += 1 # step forward
        end
        print_tree(logs, order) # print tree
    end
    return "success!"
end

GLA (generic function with 1 method)

In [10]:
order = [1, 2, 5, 6, 4, 3]
# domains
fives = ["HOSES", "LASER", "SHEET", "SNAIL", "STEER"]
fours = ["ALSO", "EARN", "HIKE", "IRON", "SAME"]
threes = ["EAT", "LET", "RUN", "SUN", "TEN", "YES"]
twos = ["BE", "IT", "NO", "US"]
# set up tree strings
logs = []
# create vars
vars = []
push!(vars, word("x1", "", deepcopy(fives), deepcopy(fives)))
push!(vars, word("x2", "", deepcopy(fours), deepcopy(fours)))
push!(vars, word("x5", "", deepcopy(twos), deepcopy(twos)))
push!(vars, word("x6", "", deepcopy(twos), deepcopy(twos)))
push!(vars, word("x4", "", deepcopy(fours), deepcopy(fours)))
push!(vars, word("x3", "", deepcopy(threes), deepcopy(threes)))
# set constrains
Rs = Dict{Vector{word},R}()
_set_constrains!(vars[1], 3, vars[2], 1, Rs)
_set_constrains!(vars[1], 5, vars[6], 1, Rs)
_set_constrains!(vars[2], 3, vars[5], 2, Rs)
_set_constrains!(vars[2], 4, vars[3], 1, Rs)
_set_constrains!(vars[6], 3, vars[5], 4, Rs)
_set_constrains!(vars[5], 3, vars[4], 2, Rs)
_set_constrains!(vars[3], 2, vars[4], 1, Rs)
GLA(order, logs, vars, Rs, SVFC)

		Domain of x1 = Any["HOSES", "LASER", "SHEET", "SNAIL", "STEER"]
		Domain of x2 = Any["ALSO", "EARN", "HIKE", "IRON", "SAME"]
		Domain of x5 = Any["BE", "IT", "NO", "US"]
		Domain of x6 = Any["BE", "IT", "NO", "US"]
		Domain of x4 = Any["ALSO", "EARN", "HIKE", "IRON", "SAME"]
		Domain of x3 = Any["EAT", "LET", "RUN", "SUN", "TEN", "YES"]
Step #1:	x1 chose HOSES from Any["LASER", "SHEET", "SNAIL", "STEER"]
x1:      ┌┄┄┄┄┄┄┬┄┄┄┄┄┄┬┄┄┄┄┄┄┬┄┄┄┄┄┬┄┄┄┄┬┄┄┄┄┬┄┄┄┄┬┄┄┄┄┬┄┄HOSES
         ┊      ┊      ┊      ┊     ┊    ┊    ┊    ┊    ┊       
x2:   [ALSO] [EARN] [HIKE] [IRON]   ┊    ┊    ┊    ┊    ┊       
                                    ┊    ┊    ┊    ┊    ┊       
x5:                                 ┊    ┊    ┊    ┊    ┊       
                                    ┊    ┊    ┊    ┊    ┊       
x6:                                 ┊    ┊    ┊    ┊    ┊       
                                    ┊    ┊    ┊    ┊    ┊       
x4:                                 ┊    ┊    ┊    ┊    ┊       
     

"inconsistent"

In [20]:
order = [1, 2, 3, 4, 5, 6]
# domains
cols = [1, 2, 3, 4, 5, 6]
# set up tree strings
logs = []
# create vars
vars = []
mutable struct Queen
    name
    row
    col
    d::Vector{Any}
end
for i = 1:6
    push!(vars, Queen("q$i", i, 0, deepcopy(cols)))
end
# set constrains
Rs = Dict{Vector{Queen},R_queen}()
for i = 1:5
    for j = i+1:6
        _set_constrains!(vars[i], vars[j], Rs)
    end
end
BACKTRACKING(order, logs, vars, Rs)

		Domain of q1 = Any[1, 2, 3, 4, 5, 6]
		Domain of q2 = Any[1, 2, 3, 4, 5, 6]
		Domain of q3 = Any[1, 2, 3, 4, 5, 6]
		Domain of q4 = Any[1, 2, 3, 4, 5, 6]
		Domain of q5 = Any[1, 2, 3, 4, 5, 6]
		Domain of q6 = Any[1, 2, 3, 4, 5, 6]
Step #1:	q1 chose 1 from Any[2, 3, 4, 5, 6]
q1:   1
       
q2:    
       
q3:    
       
q4:    
       
q5:    
       
q6:    
       
***********************************
		Domain of q1 = Any[2, 3, 4, 5, 6]
		Domain of q2 = Any[1, 2, 3, 4, 5, 6]
		Domain of q3 = Any[1, 2, 3, 4, 5, 6]
		Domain of q4 = Any[1, 2, 3, 4, 5, 6]
		Domain of q5 = Any[1, 2, 3, 4, 5, 6]
		Domain of q6 = Any[1, 2, 3, 4, 5, 6]
Step #2:	q2 chose 3 from Any[4, 5, 6]
q1:   1 ┌┄┄┬┄┓
        ┊  ┊ ┃
q2:    [1][2]3
              
q3:           
              
q4:           
              
q5:           
              
q6:           
              
***********************************
		Domain of q1 = Any[2, 3, 4, 5, 6]
		Domain of q2 = Any[4, 5, 6]
		Domain of q3 = Any[1, 2, 3, 4, 5, 6]


LoadError: BoundsError: attempt to access 6-element Vector{Any} at index [7]

In [56]:
order = [1, 2, 3, 4, 5, 6]
# domains
cols = [1, 2, 3, 4, 5, 6]
# set up tree strings
logs = []
# create vars
vars = []
for i = 1:6
    push!(vars, Queen_DVFC("q$i", i, 0, deepcopy(cols), []))
end
# set constrains
Rs = Dict{Vector{Queen_DVFC},R_queen_DVFC}()
for i = 1:5
    for j = i+1:6
        _set_constrains!(vars[i], vars[j], Rs)
    end
end
DVFC(order, logs, vars, Rs)

s = 2, so new order is [1, 2, 3, 4, 5, 6]
		Domain of q1 = Any[1, 2, 3, 4, 5, 6]
		Domain of q2 = Any[1, 2, 3, 4, 5, 6]
		Domain of q3 = Any[1, 2, 3, 4, 5, 6]
		Domain of q4 = Any[1, 2, 3, 4, 5, 6]
		Domain of q5 = Any[1, 2, 3, 4, 5, 6]
		Domain of q6 = Any[1, 2, 3, 4, 5, 6]
Step #1:	q1 chose 1 from Any[2, 3, 4, 5, 6]
s = 3, so new order is [1, 3, 2, 4, 5, 6]
q1:    ┌┄┄┬┄┄┬┄┄┬┄┄┬┄┄┬┄┄┬┄┄┬┄┄┬┄1
       ┊  ┊  ┊  ┊  ┊  ┊  ┊  ┊  ┊  
q3:   [1] ┊  ┊  ┊  ┊  ┊  ┊  ┊  ┊  
          ┊  ┊  ┊  ┊  ┊  ┊  ┊  ┊  
q2:      [1][2] ┊  ┊  ┊  ┊  ┊  ┊  
                ┊  ┊  ┊  ┊  ┊  ┊  
q4:            [1][3] ┊  ┊  ┊  ┊  
                      ┊  ┊  ┊  ┊  
q5:                  [1][4] ┊  ┊  
                            ┊  ┊  
q6:                        [1][5] 
                                  
***********************************
		Domain of q1 = Any[2, 3, 4, 5, 6]
		Domain of q3 = Any[3, 4, 5, 6]
		Domain of q2 = Any[2, 3, 4, 5, 6]
		Domain of q4 = Any[2, 4, 5, 6]
		Domain of q5 = Any[2, 3, 5, 6]
		Domain of

LoadError: BoundsError: attempt to access 4-element Vector{Any} at index [5]